In [1]:
# 复现2019年东方肝胆的论文
# # 以3年生存期为目标特征，以xgboost、rf、GBDT三个算法建模，计算特征的重要性得分并将其降序排列，分别选取前20的变量

In [2]:
import pandas as pd
import numpy as np

import sys
import re
import os
project_path = os.getcwd()

# load data

In [33]:
df_model_origin =pd.read_excel(project_path+'/data/processed_data/df_3.2.1_术前预后cox数据集.xlsx')
if 'Unnamed: 0' in df_model_origin.columns:
    df_model_origin = df_model_origin.drop(['Unnamed: 0'], axis=1)

In [34]:
df_model=df_model_origin.copy()

In [35]:
df_model.shape

(451, 62)

# 数据清洗

In [36]:
df_model.columns

Index(['id', 'sampling', 'gender', 'age', 'height', 'weight', 'BMI',
       'jaundice', 'emaciation', 'breath_disease', 'cardio_disease', 'nbdd',
       'urinary_disease', 'endocrine_disease', 'biliary_disease',
       'other_disease', 'smoke', 'drinking', 'family_history', 'blood_type',
       'WBC', 'HGB', 'PLT', 'TB', 'DB', 'TP', 'ALB', 'LG', 'AG', 'PAB', 'ALT',
       'AST', 'GT', 'ALP', 'tumor_AFP', 'tumor_CEA', 'tumor_CA19-9',
       'tumor_CA125', 'tumor_size', 'HBsAg', 'HBeAg', 'HBeAb', 'HBcAb',
       'HCVAb', 'LC', 'Gazzaniga_T', 'MSKCC', 'Blumgart_T', 'Bismuth_C',
       'PTCD_ERCP', '手术日期', 'gene_mutation', 'gene_MSI', 'TMB', 'target_death',
       'target_survival_month', 'target_1Y_death', 'target_1Y_survival',
       'target_3Y_death', 'target_3Y_survival', 'target_8Y_death',
       'target_8Y_survival'],
      dtype='object')

## 删除无关特征

In [37]:
df_model=df_model.drop(['id','手术日期','target_death','target_survival_month','target_1Y_death',
                       'target_1Y_survival','target_8Y_death','target_8Y_survival'],axis=1)

In [38]:
df_model.shape

(451, 54)

## 删除高线性相关的变量

In [39]:
# df_model=df_model.drop(['LG'],axis=1)

## 删除缺失超过50%的数据

In [40]:
# 删除列缺失超过50%
protect_col=['']
col=[i for i in df_model.columns if i not in protect_col]
for i in col:
    if df_model[i].isnull().sum()/df_model.shape[0] >= 0.5:
        print(i)
        del df_model[i]

blood_type
LG
gene_MSI
TMB


In [41]:
# 删除行缺失超过50%
df_model=df_model.reset_index(drop=True)
for i in range(df_model.shape[0]):
    if df_model.loc[i].isnull().sum()/df_model.loc[i].shape[0] >= 0.5:
        print(i)
        df_model.drop([i],axis=0,inplace=True)

194
269


## 删除分类不平衡特征

In [42]:
# 保护目标变量、tdm数据
protect_col=['sampling']
col=[i for i in df_model.columns if i not in protect_col]
for i in col:
    if df_model[i].nunique() < 2:
        print(i)
        del df_model[i]
        continue
    if df_model[i].nunique() == 2:
        # 如果分类变量中某一变量的占比超过90%，则删除该指标
        num_1 = df_model[i].value_counts()  # df一列中不同变量的数目
        num_2 = num_1.div(df_model.shape[0])  # div除法，所有元素都除以相同数值
        num_3 = num_2.max()  # 取出最大值
        if num_3 >= 0.9:
            print(i)
            del df_model[i]

breath_disease
urinary_disease
family_history


## 过滤异常值

In [43]:
# 过滤异常值，大于正常值超过100倍！
protect_col=['target_3Y_death','target_3Y_survival']
col=[i for i in df_model.columns if i not in protect_col]
for i in col:
    # 测试特征是否存在异常值
#     print(i)
    median_value= max(df_model[i].median(),1)
    max_value = df_model[i].astype(float).max()
    if max_value > 100 * median_value:
        print(i)
    df_model[i]=df_model[i].apply(lambda x: np.nan if pd.isnull(x) else 
                                            np.nan if float(x) > 100*median_value else x)

tumor_AFP


In [44]:
df_model.shape

(449, 47)

In [45]:
df_model.to_excel(project_path+'/data/reimplementation/df_2.4_数据清洗.xlsx')

# 显著性检验

In [46]:
df_model.columns

Index(['sampling', 'gender', 'age', 'height', 'weight', 'BMI', 'jaundice',
       'emaciation', 'cardio_disease', 'nbdd', 'endocrine_disease',
       'biliary_disease', 'other_disease', 'smoke', 'drinking', 'WBC', 'HGB',
       'PLT', 'TB', 'DB', 'TP', 'ALB', 'AG', 'PAB', 'ALT', 'AST', 'GT', 'ALP',
       'tumor_AFP', 'tumor_CEA', 'tumor_CA19-9', 'tumor_CA125', 'tumor_size',
       'HBsAg', 'HBeAg', 'HBeAb', 'HBcAb', 'HCVAb', 'LC', 'Gazzaniga_T',
       'MSKCC', 'Blumgart_T', 'Bismuth_C', 'PTCD_ERCP', 'gene_mutation',
       'target_3Y_death', 'target_3Y_survival'],
      dtype='object')

In [47]:
# 离散变量和连续变量
discrete_col=['sampling','gender','jaundice','emaciation','cardio_disease','nbdd','endocrine_disease','biliary_disease','other_disease',
              'smoke','drinking','HBsAg','HBeAg','HBeAb','HBcAb','HCVAb','LC','Gazzaniga_T','MSKCC','Blumgart_T',
              'Bismuth_C','PTCD_ERCP','gene_mutation']
continuous_col=[x for x in df_model.columns if x not in discrete_col]
continuous_col.remove('target_3Y_death')
continuous_col.remove('target_3Y_survival')

## 分类变量

In [48]:
from scipy.stats import chi2_contingency
feature_list=[]
y_list=[]
t_list=[]
p_list=[]
sig_list=[]
for i in discrete_col:
    print(i)
    result = chi2_contingency(pd.crosstab(df_model['target_3Y_death'],df_model[i]))
    t,p=result[0:2]
    t=round(t,2)
    p=round(p,3)
    feature_list.append(i)
    y_list.append('卡方检验')
    t_list.append(t)
    p_list.append(p)
    if p <=0.05:
        sig='显著'
    else:
        sig='不显著'
    sig_list.append(sig)

sampling
gender
jaundice
emaciation
cardio_disease
nbdd
endocrine_disease
biliary_disease
other_disease
smoke
drinking
HBsAg
HBeAg
HBeAb
HBcAb
HCVAb
LC
Gazzaniga_T
MSKCC
Blumgart_T
Bismuth_C
PTCD_ERCP
gene_mutation


In [49]:
df_discrete_corr_sig=pd.DataFrame(data={'feature':discrete_col,
                                        'method':y_list,
                                        't':t_list,
                                        'p':p_list,
                                        'result':sig_list})

In [50]:
df_discrete_corr_sig

,feature,method,t,p,result
0,sampling,卡方检验,3.23,0.072,不显著
1,gender,卡方检验,0.21,0.646,不显著
2,jaundice,卡方检验,12.24,0.000,显著
3,emaciation,卡方检验,1.11,0.574,不显著
4,cardio_disease,卡方检验,0.00,0.948,不显著
5,nbdd,卡方检验,0.58,0.446,不显著
6,endocrine_disease,卡方检验,1.14,0.285,不显著
7,biliary_disease,卡方检验,0.50,0.477,不显著
8,other_disease,卡方检验,0.96,0.326,不显著
9,smoke,卡方检验,0.20,0.654,不显著


In [51]:
discrete_relate=list(df_discrete_corr_sig[df_discrete_corr_sig.result=='显著']['feature'])

In [52]:
discrete_relate

['jaundice', 'HBcAb', 'Gazzaniga_T', 'gene_mutation']

In [53]:
list(df_discrete_corr_sig[df_discrete_corr_sig.result=='不显著']['feature'])

['sampling',
 'gender',
 'emaciation',
 'cardio_disease',
 'nbdd',
 'endocrine_disease',
 'biliary_disease',
 'other_disease',
 'smoke',
 'drinking',
 'HBsAg',
 'HBeAg',
 'HBeAb',
 'HCVAb',
 'LC',
 'MSKCC',
 'Blumgart_T',
 'Bismuth_C',
 'PTCD_ERCP']

In [54]:
# 舍弃p值大于0.5的变量，保留业务需要的不显著变量
discrete_relate.extend(['sampling', 'emaciation','endocrine_disease','biliary_disease',
                        'drinking','HBsAg','HCVAb','LC','PTCD_ERCP','MSKCC','Blumgart_T','Bismuth_C'])

## 连续变量

In [55]:
from scipy.stats import kstest,shapiro
##检验是否正态
def norm_test(data):
    if len(data) > 30:
        norm, p = kstest(data, 'norm')
    else:
        norm, p = shapiro(data)
    #print(t,p)
    if p>=0.05:
        return True
    else:
        return False

In [56]:
import scipy.stats as st
# 连续变量的显著性检验
def test2(data_b, data_p):
    if norm_test(data_b) and norm_test(data_p):
        x = 1
        y = '独立样本T检验'
        t, p = st.ttest_ind(list(data_b),list(data_p), nan_policy='omit')
    else:
        x = 0
        y = 'Mann-Whitney U检验'
        t,p = st.mannwhitneyu(list(data_b),list(data_p))
    return x,y,t,p

In [57]:
def sig_test(df_high,df_low,list1):

    feature_list=[]  # 特征列表
    y_list=[]  # 显著性检验方法
    t_list=[]  # 统计量
    p_list=[]  # p值
    result_list=[]  # 是否显著
    high_mean_list=[]
    low_mean_list=[]

    for i in list1:
        print(i)
        # 高剂量组统计
        df_high_nt=df_high[df_high[i].notnull()]
        data_high=df_high_nt[i]
        high_mean=round(data_high.mean(),2)
        
        df_low_nt=df_low[df_low[i].notnull()]
        data_low=df_low_nt[i]
        low_mean=round(data_low.mean(),2)

        # 计算高低剂量组显著性差异
        if data_high.shape[0] >= 10 and data_low.shape[0]>=10:
            # 连续变量检验
            x,y,t,p = test2(data_high, data_low)
            t=round(t,2)
            p=round(p,3)
            if p <=0.05:
                sig='显著'
            else:
                sig='不显著'
            # 显著性 
            feature_list.append(i)
            y_list.append(y)
            t_list.append(t)
            p_list.append(p)
            result_list.append(sig)
            high_mean_list.append(high_mean)
            low_mean_list.append(low_mean)

    df_result=pd.DataFrame({'特征':feature_list,
                            '高剂量均值':high_mean_list,
                            '低剂量均值':low_mean_list,
                            '检验指标':y_list,
                            't值':t_list,
                            'p值':p_list,
                            '显著性结果':result_list})
    return df_result

In [58]:
# 提取甲氨蝶呤不同标签
df_MTX_high = df_model[df_model['target_3Y_death']==1]
df_MTX_low = df_model[df_model['target_3Y_death']==0]
# 甲氨蝶呤高低剂量组PSM控制变量统计和显著性检验
df_continuous_sig = sig_test(df_MTX_high,df_MTX_low,continuous_col)
df_continuous_sig = df_continuous_sig.reset_index(drop=True)

age
height
weight
BMI
WBC
HGB
PLT
TB
DB
TP
ALB
AG
PAB
ALT
AST
GT
ALP
tumor_AFP
tumor_CEA
tumor_CA19-9
tumor_CA125
tumor_size


In [59]:
df_continuous_sig

,特征,高剂量均值,低剂量均值,检验指标,t值,p值,显著性结果
0,age,57.57,58.97,Mann-Whitney U检验,16269.0,0.115,不显著
1,height,164.97,167.75,Mann-Whitney U检验,2594.5,0.029,显著
2,weight,61.58,65.55,Mann-Whitney U检验,2637.5,0.036,显著
3,BMI,22.56,23.30,Mann-Whitney U检验,2939.0,0.143,不显著
4,WBC,6.39,6.29,Mann-Whitney U检验,10028.5,0.303,不显著
5,HGB,123.28,125.41,Mann-Whitney U检验,9373.5,0.115,不显著
6,PLT,255.61,222.89,Mann-Whitney U检验,7779.5,0.005,显著
7,TB,186.43,155.16,Mann-Whitney U检验,14439.5,0.004,显著
8,DB,145.83,138.52,Mann-Whitney U检验,9212.0,0.144,不显著
9,TP,64.78,65.77,Mann-Whitney U检验,12602.5,0.181,不显著


In [60]:
continuous_relate = list(df_continuous_sig[df_continuous_sig['显著性结果']=='显著']['特征'])

In [61]:
continuous_relate

['height', 'weight', 'PLT', 'TB', 'tumor_CA19-9', 'tumor_CA125']

In [62]:
list(df_continuous_sig[df_continuous_sig['显著性结果']=='不显著']['特征'])

['age',
 'BMI',
 'WBC',
 'HGB',
 'DB',
 'TP',
 'ALB',
 'AG',
 'PAB',
 'ALT',
 'AST',
 'GT',
 'ALP',
 'tumor_AFP',
 'tumor_CEA',
 'tumor_size']

In [63]:
# 舍弃p值大于0.5的变量，保留医学相关的变量：
continuous_relate.extend(['age','BMI','WBC','HGB','DB','ALB','GT','ALP','tumor_AFP','tumor_CEA','tumor_size'])

In [64]:
continuous_relate

['height',
 'weight',
 'PLT',
 'TB',
 'tumor_CA19-9',
 'tumor_CA125',
 'age',
 'BMI',
 'WBC',
 'HGB',
 'DB',
 'ALB',
 'GT',
 'ALP',
 'tumor_AFP',
 'tumor_CEA',
 'tumor_size']

## 提取显著特征

In [65]:
# 提取相关性特征
feature_relate=discrete_relate + continuous_relate
# 加入目标变量
feature_relate.insert(0,'target_3Y_survival')
feature_relate.insert(0,'target_3Y_death')

df_model_relate=df_model[feature_relate]

In [66]:
df_model_relate.shape

(449, 35)

In [67]:
feature_relate

['target_3Y_death',
 'target_3Y_survival',
 'jaundice',
 'HBcAb',
 'Gazzaniga_T',
 'gene_mutation',
 'sampling',
 'emaciation',
 'endocrine_disease',
 'biliary_disease',
 'drinking',
 'HBsAg',
 'HCVAb',
 'LC',
 'PTCD_ERCP',
 'MSKCC',
 'Blumgart_T',
 'Bismuth_C',
 'height',
 'weight',
 'PLT',
 'TB',
 'tumor_CA19-9',
 'tumor_CA125',
 'age',
 'BMI',
 'WBC',
 'HGB',
 'DB',
 'ALB',
 'GT',
 'ALP',
 'tumor_AFP',
 'tumor_CEA',
 'tumor_size']

In [68]:
df_model_relate.to_excel(project_path+'/data/reimplementation/df_3.3_显著性检验.xlsx')

# split data

In [69]:
df_model_split=df_model_relate.copy()

In [70]:
df_model_split.columns

Index(['target_3Y_death', 'target_3Y_survival', 'jaundice', 'HBcAb',
       'Gazzaniga_T', 'gene_mutation', 'sampling', 'emaciation',
       'endocrine_disease', 'biliary_disease', 'drinking', 'HBsAg', 'HCVAb',
       'LC', 'PTCD_ERCP', 'MSKCC', 'Blumgart_T', 'Bismuth_C', 'height',
       'weight', 'PLT', 'TB', 'tumor_CA19-9', 'tumor_CA125', 'age', 'BMI',
       'WBC', 'HGB', 'DB', 'ALB', 'GT', 'ALP', 'tumor_AFP', 'tumor_CEA',
       'tumor_size'],
      dtype='object')

In [72]:
# 离散变量和连续变量
discrete_col=['sampling','jaundice', 'HBcAb','Gazzaniga_T', 'gene_mutation','emaciation',
           'endocrine_disease', 'biliary_disease', 'drinking', 'HBsAg', 'HCVAb',
           'LC', 'PTCD_ERCP', 'MSKCC', 'Blumgart_T', 'Bismuth_C',]
continuous_col=[x for x in df_model_split.columns if x not in discrete_col]
continuous_col.remove('target_3Y_death')
continuous_col.remove('target_3Y_survival')

## 数据归一化

In [73]:
# 防止不同维特征数据差距过大，影响建模效果
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df_model_split[continuous_col]=ss.fit_transform(df_model_split[continuous_col])

## 插补数据

In [74]:
# 使用随机森林对缺失值进行插补
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
def missing_value_interpolation(df):
    df = df.reset_index(drop=True)
    
    # 提取存在缺失值的列名
    missing_list = []
    # 设定保护列
    protect_col=['target_3Y_death', 'target_3Y_survival','sampling']
    for i in df.columns:
        if i in protect_col:
            continue
        try:
            if df[i].isnull().sum()>0:
                missing_list.append(i)
        except:
            if df[i].isnull().sum()[0]>0:
                missing_list.append(i)
    missing_list_copy = missing_list.copy()
    # 用该列未缺失的值训练随机森林，然后用训练好的rf预测缺失值
    for i in range(len(missing_list)):
        name=missing_list[0]
        df_missing = df[missing_list_copy]
        # 将其他列的缺失值用0表示。
        missing_list.remove(name)
        for j in missing_list:
            df_missing[j]=df_missing[j].astype('str').apply(lambda x: 0 if x=='nan' else x)
        df_missing_is = df_missing[df_missing[name].isnull()]
        df_missing_not = df_missing[df_missing[name].notnull()]
        y = df_missing_not[name]
        x = df_missing_not.drop([name],axis=1)

        rfr = RandomForestRegressor(n_estimators=300,
                                    random_state=3)
        rfr.fit(x, y)
        #预测缺失值
        predict = rfr.predict(df_missing_is.drop([name],axis=1))
        #填补缺失值
        df.loc[df[name].isnull(),name] = predict
    return df

In [75]:
# 插补建模数据
df_model_cb=missing_value_interpolation(df_model_split)
# df_model_cb=df_model

## 划分数据集

In [77]:
# 分类随机数种子
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
# 划分训练集和测试集，比例为8:2
x = df_model_cb.drop(['target_3Y_death','target_3Y_survival','sampling'],axis=1)
y = df_model_cb['target_3Y_death']

tran_x, test_x, tran_y, test_y = train_test_split(x, y, test_size=0.2, random_state=0)

In [78]:
# 进行过采样
from imblearn.over_sampling import SMOTE,ADASYN 
from imblearn.combine import SMOTETomek
sm = SMOTE(random_state=0)
# sm=ADASYN(random_state=0)

tran_x_sm,tran_y_sm = sm.fit_resample(tran_x,tran_y)

# models

In [79]:
'''
分别训练xgboost,rf,GBDT三个模型，并提起重要性排名前20的特征
'''

'\n分别训练xgboost,rf,GBDT三个模型，并提起重要性排名前20的特征\n'

## XGBoost

In [80]:
import xgboost
xgb_model=xgboost.XGBClassifier(max_depth=4,
                        learning_rate=0.01,
                        n_estimators=500,
                        min_child_weight=0.8,
                        eta=0.1,
                        gamma=0.5,
                        reg_lambda=6,
                        subsample=0.8,
                        colsample_bytree=0.6,
                        nthread=4,
                        scale_pos_weight=1,
                        random_state=3)

xgb_model.fit(tran_x_sm,tran_y_sm)

importance = xgb_model.feature_importances_
print(importance)

[0.04332235 0.03616276 0.03302703 0.0206549  0.02891917 0.02146459
 0.02124949 0.02677844 0.01510802 0.01153086 0.03085782 0.02217596
 0.02868218 0.02461437 0.01907788 0.10975234 0.09710968 0.04801979
 0.02186425 0.01748527 0.02589551 0.02617571 0.05754536 0.0249344
 0.04208746 0.02405325 0.01683735 0.01629894 0.01821223 0.02971992
 0.01669567 0.02368703]


In [81]:
df_importance= pd.DataFrame(data={'特征':x.columns,'重要性评分':importance})
df_importance['重要性评分']=df_importance['重要性评分'].apply(lambda x: round(x,3))
df_importance=df_importance.sort_values(['重要性评分'],ascending=False)
df_importance=df_importance.reset_index(drop=True)

In [82]:
df_importance

,特征,重要性评分
0,height,0.110
1,weight,0.097
2,BMI,0.058
3,PLT,0.048
4,jaundice,0.043
5,HGB,0.042
6,HBcAb,0.036
7,Gazzaniga_T,0.033
8,LC,0.031
9,tumor_AFP,0.030


In [83]:
# 取前20个变量
col_xgb_20=df_importance.loc[:19,'特征'].to_list()

In [84]:
col_xgb_20

['height',
 'weight',
 'BMI',
 'PLT',
 'jaundice',
 'HGB',
 'HBcAb',
 'Gazzaniga_T',
 'LC',
 'tumor_AFP',
 'MSKCC',
 'emaciation',
 'drinking',
 'tumor_CA125',
 'age',
 'Blumgart_T',
 'WBC',
 'tumor_size',
 'DB',
 'TB']

## RF

In [85]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
# 列出参数列表
tree_grid_parameter = {'n_estimators': list((10, 50, 100, 150, 200))}
# 进行参数的搜索组合
grid = GridSearchCV(RandomForestClassifier(), param_grid=tree_grid_parameter, cv=3)
# 根据已有数据去拟合随机森林模型
grid.fit(tran_x_sm, tran_y_sm)
rf_model = RandomForestClassifier(n_estimators=grid.best_params_['n_estimators'],
                            max_depth=8,
                            random_state=3)
rf_model.fit(tran_x_sm, tran_y_sm)

importance = rf_model.feature_importances_
print(importance)

[0.02644076 0.01450273 0.01930959 0.00147408 0.01964853 0.00429823
 0.01054825 0.00835235 0.00466172 0.00183167 0.00545924 0.01658176
 0.00730438 0.00703659 0.01608885 0.16387379 0.12275222 0.07210675
 0.02953401 0.0178169  0.02381514 0.02677114 0.1073297  0.02985323
 0.06916921 0.03186573 0.0219253  0.02789082 0.02312495 0.02769214
 0.01676205 0.0241782 ]


In [86]:
df_importance= pd.DataFrame(data={'特征':x.columns,'重要性评分':importance})
df_importance['重要性评分']=df_importance['重要性评分'].apply(lambda x: round(x,3))
df_importance=df_importance.sort_values(['重要性评分'],ascending=False)
df_importance=df_importance.reset_index(drop=True)

In [87]:
df_importance

,特征,重要性评分
0,height,0.164
1,weight,0.123
2,BMI,0.107
3,PLT,0.072
4,HGB,0.069
5,DB,0.032
6,WBC,0.030
7,TB,0.030
8,tumor_AFP,0.028
9,GT,0.028


In [88]:
# 取前20个变量
col_rf_20=df_importance.loc[:19,'特征'].to_list()

In [89]:
col_rf_20

['height',
 'weight',
 'BMI',
 'PLT',
 'HGB',
 'DB',
 'WBC',
 'TB',
 'tumor_AFP',
 'GT',
 'age',
 'jaundice',
 'tumor_size',
 'tumor_CA125',
 'ALP',
 'ALB',
 'emaciation',
 'Gazzaniga_T',
 'tumor_CA19-9',
 'PTCD_ERCP']

## GBDT

In [90]:
# GBDT
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
gbdt_model = GradientBoostingClassifier(n_estimators=300,
                            learning_rate=0.1,
                            max_depth=8,
                            subsample=0.4,
                            random_state=3)
gbdt_model.fit(tran_x_sm, tran_y_sm)

importance = gbdt_model.feature_importances_
print(importance)

[0.0239619  0.01449256 0.01308403 0.00235871 0.00913464 0.00533108
 0.02085612 0.01419789 0.00202577 0.00816463 0.01812573 0.01171677
 0.00597555 0.0006964  0.02022065 0.09562357 0.14020492 0.06766925
 0.02487712 0.01868793 0.04744826 0.0337056  0.11847161 0.01987921
 0.07894363 0.02996075 0.01807823 0.02555337 0.01711569 0.02979189
 0.02052655 0.04312   ]


In [91]:
df_importance= pd.DataFrame(data={'特征':x.columns,'重要性评分':importance})
df_importance['重要性评分']=df_importance['重要性评分'].apply(lambda x: round(x,3))
df_importance=df_importance.sort_values(['重要性评分'],ascending=False)
df_importance=df_importance.reset_index(drop=True)

In [92]:
df_importance

,特征,重要性评分
0,weight,0.140
1,BMI,0.118
2,height,0.096
3,HGB,0.079
4,PLT,0.068
5,tumor_CA125,0.047
6,tumor_size,0.043
7,age,0.034
8,tumor_AFP,0.030
9,DB,0.030


In [93]:
# 取前20个变量
col_gbdt_20=df_importance.loc[:19,'特征'].to_list()

In [94]:
col_gbdt_20

['weight',
 'BMI',
 'height',
 'HGB',
 'PLT',
 'tumor_CA125',
 'tumor_size',
 'age',
 'tumor_AFP',
 'DB',
 'GT',
 'TB',
 'jaundice',
 'tumor_CEA',
 'biliary_disease',
 'WBC',
 'Bismuth_C',
 'tumor_CA19-9',
 'ALB',
 'LC']

## 取重要变量交集

In [95]:
df_col_importance=pd.DataFrame(data={'xgb_col':col_xgb_20,
                                    'rf_col':col_rf_20,
                                    'gbdt_col':col_gbdt_20})

In [96]:
df_col_importance

,xgb_col,rf_col,gbdt_col
0,height,height,weight
1,weight,weight,BMI
2,BMI,BMI,height
3,PLT,PLT,HGB
4,jaundice,HGB,PLT
5,HGB,DB,tumor_CA125
6,HBcAb,WBC,tumor_size
7,Gazzaniga_T,TB,age
8,LC,tumor_AFP,tumor_AFP
9,tumor_AFP,GT,DB


In [212]:
df_col_importance.to_excel(project_path+'/data/reimplementation/df_5.4_top20重要变量.xlsx')

In [97]:
# col_xgb_20, col_rf_20, col_gbdt_20，得到10个交集变量
col_importance_merge=[i for i in col_xgb_20 if i in col_rf_20]
col_importance_merge=[i for i in col_gbdt_20 if i in col_importance_merge]

In [98]:
col_importance_merge

['weight',
 'BMI',
 'height',
 'HGB',
 'PLT',
 'tumor_CA125',
 'tumor_size',
 'age',
 'tumor_AFP',
 'DB',
 'TB',
 'jaundice',
 'WBC']

In [99]:
## 插入目标变量
col_importance_merge.insert(0,'sampling')
col_importance_merge.extend(['target_3Y_death', 'target_3Y_survival'])

In [100]:
col_importance_merge

['sampling',
 'weight',
 'BMI',
 'height',
 'HGB',
 'PLT',
 'tumor_CA125',
 'tumor_size',
 'age',
 'tumor_AFP',
 'DB',
 'TB',
 'jaundice',
 'WBC',
 'target_3Y_death',
 'target_3Y_survival']

In [101]:
df_model_merge=df_model_relate[col_importance_merge]

In [102]:
df_model_merge.to_excel(project_path+'/data/reimplementation/df_5.4_取重要变量交集.xlsx')

# 计算C_index

## 所有变量

In [227]:
df_model_cox_all=df_model_relate[df_model_relate.target_3Y_survival.notnull() & df_model_relate.sampling.notnull()]

In [228]:
df_model_cox_all.shape

(361, 35)

In [229]:
df_model_cox_all.columns

Index(['target_3Y_death', 'target_3Y_survival', 'jaundice', 'HBcAb',
       'Gazzaniga_T', 'gene_mutation', 'sampling', 'emaciation',
       'endocrine_disease', 'biliary_disease', 'drinking', 'HBsAg', 'HCVAb',
       'LC', 'PTCD_ERCP', 'MSKCC', 'Blumgart_T', 'Bismuth_C', 'height',
       'weight', 'PLT', 'TB', 'tumor_CA19-9', 'tumor_CA125', 'age', 'BMI',
       'WBC', 'HGB', 'DB', 'ALB', 'GT', 'ALP', 'tumor_AFP', 'tumor_CEA',
       'tumor_size'],
      dtype='object')

### 查看缺失率

In [230]:
# 删除列缺失超过50%
for i in df_model_cox_all.columns:
    percent_col= df_model_cox_all[i].isnull().sum()/df_model_cox_all.shape[0]
    percent_col="%.2f%%" % (percent_col * 100)
    print('每列缺失率:',i,percent_col)

每列缺失率: target_3Y_death 0.00%
每列缺失率: target_3Y_survival 0.00%
每列缺失率: jaundice 0.00%
每列缺失率: HBcAb 13.02%
每列缺失率: Gazzaniga_T 6.65%
每列缺失率: gene_mutation 0.00%
每列缺失率: sampling 0.00%
每列缺失率: emaciation 14.40%
每列缺失率: endocrine_disease 0.28%
每列缺失率: biliary_disease 0.28%
每列缺失率: drinking 4.16%
每列缺失率: HBsAg 8.59%
每列缺失率: HCVAb 13.30%
每列缺失率: LC 22.16%
每列缺失率: PTCD_ERCP 5.26%
每列缺失率: MSKCC 6.65%
每列缺失率: Blumgart_T 6.65%
每列缺失率: Bismuth_C 6.37%
每列缺失率: height 38.50%
每列缺失率: weight 38.50%
每列缺失率: PLT 15.79%
每列缺失率: TB 0.83%
每列缺失率: tumor_CA19-9 1.39%
每列缺失率: tumor_CA125 33.24%
每列缺失率: age 0.28%
每列缺失率: BMI 38.50%
每列缺失率: WBC 14.13%
每列缺失率: HGB 14.96%
每列缺失率: DB 13.85%
每列缺失率: ALB 3.60%
每列缺失率: GT 30.19%
每列缺失率: ALP 32.69%
每列缺失率: tumor_AFP 4.71%
每列缺失率: tumor_CEA 2.49%
每列缺失率: tumor_size 22.71%


In [231]:
# 删除数据缺失超过30%的重要变量，否则变量数超过数据量，cox无法运行
missing_col=['height','weight','BMI','tumor_CA125','GT','ALP','tumor_size']
df_model_cox_all=df_model_cox_all.drop(missing_col,axis=1)
# 删除损害收敛性的变量
df_model_cox_all=df_model_cox_all.drop(['gene_mutation', 'HCVAb','LC','sampling'],axis=1)

### dataset for cox-all

In [232]:
# 计算3年生存期分期的c_index_all
for i in df_model_cox_all.columns:
    df_model_cox_all = df_model_cox_all[df_model_cox_all[i].notnull()]

In [233]:
df_model_cox_all.sampling.value_counts()

AttributeError: 'DataFrame' object has no attribute 'sampling'

In [234]:
df_model_cox_all.shape

(190, 24)

In [235]:
df_model_cox_all.to_excel(project_path+'/data/reimplementation/df_6.1.2_dataset_cox_all.xlsx')

### c_index_all

In [236]:
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index

cph = CoxPHFitter()
cph.fit(df_model_cox_all, duration_col='target_3Y_survival',event_col='target_3Y_death',show_progress=True)
c_index_all = cph.concordance_index_
print(c_index_all)

Iteration 1: norm_delta = 1.92861, step_size = 0.9000, log_lik = -673.67474, newton_decrement = 35.22008, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.51719, step_size = 0.9000, log_lik = -643.60197, newton_decrement = 1.62422, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.06271, step_size = 0.9000, log_lik = -641.94065, newton_decrement = 0.05564, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00163, step_size = 1.0000, log_lik = -641.88373, newton_decrement = 0.00009, seconds_since_start = 0.0
Iteration 5: norm_delta = 0.00000, step_size = 1.0000, log_lik = -641.88364, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 5 iterations.
0.708030523255814


## ML筛选的变量

In [237]:
df_model_cox_ML=df_model_merge[df_model_merge.target_3Y_survival.notnull() & df_model_merge.sampling.notnull()]

In [238]:
df_model_cox_ML.columns

Index(['sampling', 'weight', 'BMI', 'height', 'HGB', 'PLT', 'tumor_CA125',
       'tumor_size', 'age', 'tumor_AFP', 'DB', 'TB', 'jaundice', 'WBC',
       'target_3Y_death', 'target_3Y_survival'],
      dtype='object')

In [239]:
df_model_cox_ML.shape

(361, 16)

### 查看缺失率

In [240]:
# 删除列缺失超过50%
for i in df_model_cox_ML.columns:
    percent_col= df_model_cox_ML[i].isnull().sum()/df_model_cox_ML.shape[0]
    percent_col="%.2f%%" % (percent_col * 100)
    print('每列缺失率:',i,percent_col)

每列缺失率: sampling 0.00%
每列缺失率: weight 38.50%
每列缺失率: BMI 38.50%
每列缺失率: height 38.50%
每列缺失率: HGB 14.96%
每列缺失率: PLT 15.79%
每列缺失率: tumor_CA125 33.24%
每列缺失率: tumor_size 22.71%
每列缺失率: age 0.28%
每列缺失率: tumor_AFP 4.71%
每列缺失率: DB 13.85%
每列缺失率: TB 0.83%
每列缺失率: jaundice 0.00%
每列缺失率: WBC 14.13%
每列缺失率: target_3Y_death 0.00%
每列缺失率: target_3Y_survival 0.00%


In [241]:
# 删除数据缺失超过30%的重要变量，否则变量数超过数据量，cox无法运行
missing_col=['height','weight','BMI','tumor_CA125','tumor_size']
df_model_cox_ML=df_model_cox_ML.drop(missing_col,axis=1)
# 删除损害收敛性的变量
df_model_cox_ML=df_model_cox_ML.drop(['sampling'],axis=1)

### dataset for cox-ML

In [242]:
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
# 计算3年生存期分期的c_index_all
for i in df_model_cox_ML.columns:
    df_model_cox_ML = df_model_cox_ML[df_model_cox_ML[i].notnull()]

In [243]:
df_model_cox_ML.sampling.value_counts()

AttributeError: 'DataFrame' object has no attribute 'sampling'

In [244]:
df_model_cox_ML.shape

(284, 10)

In [245]:
df_model_cox_ML.to_excel(project_path+'/data/reimplementation/df_6.2.2_dataset_cox_ML.xlsx')

### c_index_ML

In [247]:
cph = CoxPHFitter()
cph.fit(df_model_cox_ML, duration_col='target_3Y_survival',event_col='target_3Y_death',show_progress=True)
c_index_all = cph.concordance_index_
print(c_index_all)

Iteration 1: norm_delta = 1.25022, step_size = 0.9000, log_lik = -1151.69961, newton_decrement = 8.26959, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.07191, step_size = 0.9000, log_lik = -1143.52171, newton_decrement = 0.25703, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00953, step_size = 0.9000, log_lik = -1143.26033, newton_decrement = 0.00608, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00009, step_size = 1.0000, log_lik = -1143.25421, newton_decrement = 0.00000, seconds_since_start = 0.0
Iteration 5: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1143.25421, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 5 iterations.
0.5632529319469192


## SPSS：backward筛选的变量

In [254]:
col_backward=['Gazzaniga_T','endocrine_disease','biliary_disease','TB','tumor_CA19-9','DB']
# col_backward.insert(0,'sampling')
col_backward.extend(['target_3Y_death','target_3Y_survival'])

In [255]:
df_model_cox_back=df_model_relate[col_backward]
df_model_cox_back=df_model_cox_back[df_model_cox_back.target_3Y_survival.notnull()]

### 查看缺失率

In [256]:
# 删除列缺失超过50%
for i in df_model_cox_back.columns:
    percent_col= df_model_cox_back[i].isnull().sum()/df_model_cox_back.shape[0]
    percent_col="%.2f%%" % (percent_col * 100)
    print('每列缺失率:',i,percent_col)

每列缺失率: Gazzaniga_T 6.65%
每列缺失率: endocrine_disease 0.28%
每列缺失率: biliary_disease 0.28%
每列缺失率: TB 0.83%
每列缺失率: tumor_CA19-9 1.39%
每列缺失率: DB 13.85%
每列缺失率: target_3Y_death 0.00%
每列缺失率: target_3Y_survival 0.00%


### dataset for cox-back

In [257]:
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
# 计算3年生存期分期的c_index_all
for i in df_model_cox_back.columns:
    df_model_cox_back = df_model_cox_back[df_model_cox_back[i].notnull()]

In [258]:
df_model_cox_back.shape

(285, 8)

In [259]:
df_model_cox_back.sampling.value_counts()

AttributeError: 'DataFrame' object has no attribute 'sampling'

In [260]:
df_model_cox_back.to_excel(project_path+'/data/reimplementation/df_6.3.2_dataset_cox_back.xlsx')

### c_index_back

In [261]:
cph = CoxPHFitter()
cph.fit(df_model_cox_back, duration_col='target_3Y_survival',event_col='target_3Y_death',show_progress=True)
c_index_all = cph.concordance_index_
print(c_index_all)

Iteration 1: norm_delta = 1.03751, step_size = 0.9000, log_lik = -1157.35209, newton_decrement = 46.52107, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.44369, step_size = 0.9000, log_lik = -1117.02331, newton_decrement = 1.19085, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.03253, step_size = 0.9000, log_lik = -1115.83957, newton_decrement = 0.01537, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00006, step_size = 1.0000, log_lik = -1115.82415, newton_decrement = 0.00000, seconds_since_start = 0.0
Iteration 5: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1115.82415, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 5 iterations.
0.695561646723947


## ML+SPSS筛选的变量

In [262]:
# 人工设定3个
col_ML_back=['Gazzaniga_T','jaundice','tumor_CA19-9']
col_ML_back.insert(0,'sampling')
col_ML_back.extend(['target_3Y_death','target_3Y_survival'])

In [263]:
df_model_cox_ML_back=df_model_relate[col_ML_back]
df_model_cox_ML_back=df_model_cox_ML_back[df_model_cox_ML_back.target_3Y_survival.notnull() ]

### 查看缺失率

In [264]:
# 删除列缺失超过50%
for i in df_model_cox_ML_back.columns:
    percent_col= df_model_cox_ML_back[i].isnull().sum()/df_model_cox_ML_back.shape[0]
    percent_col="%.2f%%" % (percent_col * 100)
    print('每列缺失率:',i,percent_col)

每列缺失率: sampling 0.00%
每列缺失率: Gazzaniga_T 6.65%
每列缺失率: jaundice 0.00%
每列缺失率: tumor_CA19-9 1.39%
每列缺失率: target_3Y_death 0.00%
每列缺失率: target_3Y_survival 0.00%


### dataset for cox-ML-back

In [265]:
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
# 计算3年生存期分期的c_index_all
for i in df_model_cox_ML_back.columns:
    df_model_cox_ML_back = df_model_cox_ML_back[df_model_cox_ML_back[i].notnull()]

In [266]:
df_model_cox_ML_back.shape

(332, 6)

In [267]:
df_model_cox_ML_back.sampling.value_counts()

1    329
0      3
Name: sampling, dtype: int64

In [268]:
df_model_cox_ML_back.to_excel(project_path+'/data/reimplementation/df_6.3.2_dataset_cox_ML_back.xlsx')

### c_index_ML_back

In [269]:
cph = CoxPHFitter()
cph.fit(df_model_cox_ML_back, duration_col='target_3Y_survival',event_col='target_3Y_death',cluster_col='sampling',show_progress=True)
c_index_all = cph.concordance_index_
print(c_index_all)

Iteration 1: norm_delta = 0.52886, step_size = 0.9000, log_lik = -1258.31151, newton_decrement = 30.75140, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.08326, step_size = 0.9000, log_lik = -1230.97915, newton_decrement = 0.66026, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.01283, step_size = 0.9000, log_lik = -1230.31219, newton_decrement = 0.01344, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00015, step_size = 1.0000, log_lik = -1230.29866, newton_decrement = 0.00000, seconds_since_start = 0.0
Iteration 5: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1230.29866, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 5 iterations.
0.6594318803020793


In [270]:
cph.print_summary(columns=['coef'])

model,lifelines.CoxPHFitter
duration col,'target_3Y_survival'
event col,'target_3Y_death'
cluster col,'sampling'
robust variance,True
baseline estimation,breslow
number of observations,332
number of events observed,237
partial log-likelihood,-1230.30
time fit was run,2022-03-08 02:41:07 UTC
,coef


# 计算BS

In [470]:
# 计算3年生存期的BS
from sksurv.metrics import brier_score
from sksurv.datasets import load_gbsg2
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import brier_score
from sksurv.preprocessing import OneHotEncoder

ContextualVersionConflict: (scikit-learn 0.24.2 (c:\users\administrator\appdata\roaming\python\python36\site-packages), Requirement.parse('scikit-learn<0.24,>=0.22.0'), {'scikit-survival'})

## 所有变量

In [1]:
df_model_cox_all=pd.read_excel(project_path+'/data/reimplementation/df_6.1.2_dataset_cox_all.xlsx')
if 'Unnamed: 0' in df_model_cox_all.columns:
    df_model_cox_all = df_model_cox_all.drop(['Unnamed: 0'], axis=1)

NameError: name 'pd' is not defined

In [ ]:
# 转换成boolean值
df_model_cox_all['target_death_binary']=df_model_cox_all.target_3Y_death.astype(bool)
# 构建BStrain_data和test_data
bs_y = [(d,s) for d,s in zip(df_model_cox_all.target_death_binary,df_model_cox_all.target_3Y_survival)]
bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

In [471]:
df_model_cox_all.columns

Index(['target_3Y_death', 'target_3Y_survival', 'jaundice', 'HBcAb',
       'sampling', 'gender', 'emaciation', 'cardio_disease',
       'endocrine_disease', 'smoke', 'drinking', 'HBsAg', 'PTCD_ERCP', 'MSKCC',
       'Bismuth_C', 'PLT', 'tumor_CA19-9', 'tumor_CA125', 'age', 'BMI', 'WBC',
       'HGB', 'TP', 'ALB', 'ALT', 'AST', 'tumor_AFP', 'tumor_CEA',
       'tumor_size'],
      dtype='object')

In [472]:
# 术前3年BS_all
bs_x_all = df_model_cox_all.drop(['target_3Y_death','target_3Y_survival','sampling'],axis=1)
est_all = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_all, bs_y)
survs_all = est_all.predict_survival_function(bs_x_all)
preds_all = [fn(10) for fn in survs_all]
times, bs_3Y_all = brier_score(bs_y, bs_y, preds_all, 10)
print(bs_3Y_all)

KeyError: "['target_3Y_death' 'target_3Y_survival' 'sampling'] not found in axis"

## ML筛选的变量

In [ ]:
df_model_cox_ML=pd.read_excel(project+'/data/reimplementation/df_6.2.2_dataset_cox_ML.xlsx')
if 'Unnamed: 0' in df_model_cox_ML.columns:
    df_model_cox_ML = df_model_cox_ML.drop(['Unnamed: 0'], axis=1)

In [ ]:
# 转换成布尔值
df_model_cox_ML['target_death_binary']=df_model_cox_ML.target_3Y_death.astype(bool)
# 构建BStrain_data和test_data
bs_y = [(d,s) for d,s in zip(df_model_cox_ML.target_death_binary,df_model_cox_ML.target_3Y_survival)]
bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

In [ ]:
# 术前3年BS_ML
bs_x_ML = df_model_cox_ML.drop(['target_3Y_death','target_3Y_survival','sampling'],axis=1)
est_ML = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_ML, bs_y)
survs_ML = est_ML.predict_survival_function(bs_x_ML)
preds_ML = [fn(10) for fn in survs_ML]
times, bs_3Y_ML = brier_score(bs_y, bs_y, preds_ML, 10)
print(bs_3Y_ML)

## SPSS:backward筛选的变量

In [ ]:
df_model_cox_backward=pd.read_excel(project+'/data/reimplementation/df_6.3.2_dataset_cox_back.xlsx')
if 'Unnamed: 0' in df_model_cox_backward.columns:
    df_model_cox_backward = df_model_cox_backward.drop(['Unnamed: 0'], axis=1)

In [ ]:
# 转换成布尔值
df_model_cox_backward['target_death_binary']=df_model_cox_backward.target_3Y_death.astype(bool)
# 构建BStrain_data和test_data
bs_y = [(d,s) for d,s in zip(df_model_cox_backward.target_death_binary,df_model_cox_backward.target_3Y_survival)]
bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

In [ ]:
# 术前3年BS_backward
bs_x_backward = df_model_cox_backward.drop(['target_3Y_death','target_3Y_survival','sampling'],axis=1)
est_backward = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_backward, bs_y)
survs_backward = est_backward.predict_survival_function(bs_x_backward)
preds_backward = [fn(10) for fn in survs_backward]
times, bs_3Y_backward = brier_score(bs_y, bs_y, preds_backward, 10)
print(bs_3Y_backward)

## ML+SPSS筛选的变量

In [ ]:
df_model_cox_ML_back=pd.read_excel(project+'/data/reimplementation/df_6.2.2_dataset_cox_ML.xlsx')
if 'Unnamed: 0' in df_model_cox_ML_back.columns:
    df_model_cox_ML_back = df_model_cox_ML_back.drop(['Unnamed: 0'], axis=1)

In [ ]:
# 转换成布尔值
df_model_cox_ML_back['target_death_binary']=df_model_cox_ML_back.target_3Y_death.astype(bool)
# 构建BStrain_data和test_data
bs_y = [(d,s) for d,s in zip(df_model_cox_ML_back.target_death_binary,df_model_cox_ML_back.target_3Y_survival)]
bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

In [ ]:
# 术前3年BS_ML_back
bs_x_ML_back = df_model_cox_ML_back.drop(['target_3Y_death','target_3Y_survival','sampling'],axis=1)
est_ML_back = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_ML_back, bs_y)
survs_ML_back = est_ML_back.predict_survival_function(bs_x_ML_back)
preds_ML_back = [fn(10) for fn in survs_ML_back]
times, bs_3Y_ML_back = brier_score(bs_y, bs_y, preds_ML_back, 10)
print(bs_3Y_ML_back)

## 保存BS值

In [474]:
df_bs = pd.DataFrame(data={'bs_3Y_all':bs_3Y_all,
                          'bs_3Y_ML':bs_3Y_ML,
                          'bs_3Y_backward':bs_3Y_backward,
                          'bs_3Y_ML_back':bs_3Y_ML_back})
df_bs.to_excel(project_path+'/data/implementation/df_bs.xlsx')

NameError: name 'bs_3Y_all' is not defined

# 构建风险评分模型

In [279]:
# 根据ML+SPSS筛选的变量构建风险模型:'PLT','jaundice','Gazzaniga_T'
import math
# 计算风险评分
df_model_risk_before=df_model_relate.reset_index(drop=True)
# 缺失项补充为0
discrete_col=['Gazzaniga_T','jaundice']
df_model_risk_before[discrete_col]=df_model_risk_before[discrete_col].fillna(0)
cotinuous_col=['tumor_CA19-9']
df_model_risk_before[cotinuous_col]=df_model_risk_before[cotinuous_col].fillna(1)
for i in range(df_model_risk_before.shape[0]):
    gazz_value=float(df_model_risk_before.loc[i,'Gazzaniga_T'])
    jaundice_value=float(df_model_risk_before.loc[i,'jaundice'])
    value_199=float(df_model_risk_before.loc[i,'tumor_CA19-9'])
#     tb_value=float(df_model_risk_before.loc[i,'TB'])
#     db_value=float(df_model_risk_before.loc[i,'DB'])
#     plt=float(df_model_risk_before.loc[i,'PLT'])
    # 术前1年生存期risk_score计算
    df_model_risk_before.loc[i,'risk_score']=math.log(value_199)*0.001+0.555*jaundice_value+0.323*gazz_value
#     df_model_risk_before.loc[i,'risk_score']=math.log(plt)*0.002+0.41*jaundice_value+0.41*gazz_value

In [280]:
df_model_risk_before.risk_score.describe()

count    449.000000
mean       1.014825
std        0.412648
min        0.000000
25%        0.881526
50%        0.884004
75%        1.207093
max        1.853908
Name: risk_score, dtype: float64

## 分期

In [281]:
# 术前1年生存期风险评分分期
df_model_risk_before['staging']=df_model_risk_before.risk_score.apply(lambda x: 1 if x<=0.5 else
                                                                     2 if 0.5<x<=1 else
                                                                     3 if 1<x<=1.5 else
                                                                     4 if x>1.5 else np.nan)

In [282]:
df_model_risk_before.staging.value_counts()

2    262
4    100
3     47
1     40
Name: staging, dtype: int64

## 计算C_index

### c_index_us

In [283]:
df_C_index_us.staging.value_counts()

2.0    171
4.0     93
3.0     42
1.0     26
Name: staging, dtype: int64

In [284]:
df_C_index_us.isnull().any()

staging                True
target_3Y_death       False
target_3Y_survival    False
dtype: bool

In [285]:
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
# 计算1年生存期分期的c_index_us
df_C_index_us = df_model_risk_before[['staging','target_3Y_death','target_3Y_survival']]
df_C_index_us = df_C_index_us[df_C_index_us.target_3Y_survival.notnull()]
cph = CoxPHFitter()
cph.fit(df_C_index_us, duration_col='target_3Y_survival', event_col='target_3Y_death', show_progress=True)
c_index_us = cph.concordance_index_
print(c_index_us)

Iteration 1: norm_delta = 0.39896, step_size = 0.9000, log_lik = -1395.76908, newton_decrement = 18.83355, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00501, step_size = 0.9000, log_lik = -1378.14352, newton_decrement = 0.00346, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00050, step_size = 0.9000, log_lik = -1378.14009, newton_decrement = 0.00003, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1378.14005, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.
0.6146029356417012


### c_index_gazz

In [286]:
# 计算1年生存期分期的c_index_gazz
df_C_index_gazz = df_model_risk_before[['Gazzaniga_T','target_3Y_death','target_3Y_survival']]
df_C_index_gazz = df_C_index_gazz[df_C_index_gazz.target_3Y_survival.notnull()]
cph = CoxPHFitter()
cph.fit(df_C_index_gazz, duration_col='target_3Y_survival', event_col='target_3Y_death', show_progress=True)
c_index_gazz = cph.concordance_index_
print(c_index_gazz)

Iteration 1: norm_delta = 0.36613, step_size = 0.9000, log_lik = -1395.76908, newton_decrement = 12.63737, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00967, step_size = 0.9000, log_lik = -1384.28619, newton_decrement = 0.01135, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00099, step_size = 0.9000, log_lik = -1384.27494, newton_decrement = 0.00012, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1384.27482, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.
0.5834502343723269


### c_index_mskcc

In [287]:
# 计算1年生存期分期的c_index_mskcc
df_C_index_mskcc = df_model_risk_before[['MSKCC','target_3Y_death','target_3Y_survival']]
df_C_index_mskcc = df_C_index_mskcc[df_C_index_mskcc.target_3Y_survival.notnull() & df_C_index_mskcc.MSKCC.notnull()]
cph = CoxPHFitter()
cph.fit(df_C_index_mskcc, duration_col='target_3Y_survival', event_col='target_3Y_death', show_progress=True)
c_index_mskcc = cph.concordance_index_
print(c_index_mskcc)

Iteration 1: norm_delta = 0.40187, step_size = 0.9000, log_lik = -1273.65809, newton_decrement = 15.65977, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00933, step_size = 0.9000, log_lik = -1259.42443, newton_decrement = 0.01060, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00095, step_size = 0.9000, log_lik = -1259.41393, newton_decrement = 0.00011, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1259.41382, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.
0.6093805414351579


### c_index_blum

In [288]:
# 计算1年生存期分期的c_index_blumgart
df_C_index_blumgart = df_model_risk_before[['Blumgart_T','target_3Y_death','target_3Y_survival']]
df_C_index_blumgart = df_C_index_blumgart[df_C_index_blumgart.target_3Y_survival.notnull() & df_C_index_blumgart.Blumgart_T.notnull()]
cph = CoxPHFitter()
cph.fit(df_C_index_blumgart, duration_col='target_3Y_survival', event_col='target_3Y_death', show_progress=True)
c_index_blumgart = cph.concordance_index_
print(c_index_blumgart)

Iteration 1: norm_delta = 0.40676, step_size = 0.9000, log_lik = -1273.65809, newton_decrement = 16.20180, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00685, step_size = 0.9000, log_lik = -1258.85878, newton_decrement = 0.00567, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00069, step_size = 0.9000, log_lik = -1258.85316, newton_decrement = 0.00006, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1258.85310, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.
0.6107104858730347


In [289]:
df_model_risk_before.to_excel(project_path+'/data/reimplementation/df_8.2.4_分期c_index.xlsx')

## 计算BS

In [ ]:
# 术前预后1年生存期BS，生成布尔型True、False
df_model_bs_3Y=df_model_risk_before[df_model_risk_before.target_3Y_survival.notnull() & 
                                    df_model_risk_before.MSKCC.notnull()]
df_model_bs_3Y['target_death_binary']=df_model_bs_3Y.target_3Y_death.astype(bool)
# 构建BStrain_data和test_data
bs_y = [(d,s) for d,s in zip(df_model_bs_3Y.target_death_binary,df_model_bs_3Y.target_3Y_survival)]
bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

### bs_us

In [901]:
# 术前1年BS_us
bs_x_us = df_model_bs_3Y[['staging']]
est_us = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_us, bs_y)
survs_us = est_us.predict_survival_function(bs_x_us)
preds_us = [fn(10) for fn in survs_us]
times, bs_3Y_us = brier_score(bs_y, bs_y, preds_us, 10)
print(bs_3Y_us)

NameError: name 'df_model_bs_3Y' is not defined

### bs_gazz

In [ ]:
# 术前1年BS_gazz
bs_x_gazz=df_model_bs_3Y[['Gazzaniga_T']]
est_gazz = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_gazz, bs_y)
survs_gazz = est_gazz.predict_survival_function(bs_x_gazz)
preds_gazz = [fn(10) for fn in survs_gazz]
times, bs_3Y_gazz = brier_score(bs_y, bs_y, preds_gazz, 10)
print(bs_3Y_gazz)

### bs_mskcc

In [ ]:
# 术前1年BS_mskcc
bs_x_mskcc=df_model_bs_3Y[['MSKCC']]
est_mskcc = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_mskcc, bs_y)
survs_mskcc = est_mskcc.predict_survival_function(bs_x_mskcc)
preds_mskcc = [fn(10) for fn in survs_mskcc]
times, bs_3Y_mskcc = brier_score(bs_y, bs_y, preds_mskcc, 10)
print(bs_3Y_mskcc)

### bs_blum

In [ ]:
# 术前1年BS_blumgart
bs_x_blumgart=df_model_bs_3Y[['Blumgart_T']]
est_blumgart = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_blumgart, bs_y)
survs_blumgart = est_blumgart.predict_survival_function(bs_x_blumgart)
preds_blumgart = [fn(10) for fn in survs_blumgart]
times, bs_3Y_blumgart = brier_score(bs_y, bs_y, preds_blumgart, 10)
print(bs_3Y_blumgart)